In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2


In [2]:
print(os.listdir('own_dataset/1d10/train'))

['8', '5', '1', '7', '4', '0', '3', '6', '2', '9']


In [3]:
train_path= ('own_dataset/1d10/train')
test_path= ('own_dataset/1d10/test')

In [4]:
batch_size_train=50  #,batch_size= batch_size_train
batch_size_valid=50
targetsize= 80
datagen=ImageDataGenerator(rotation_range=180,height_shift_range=0.1,rescale=1.0/255
                           ,zoom_range=[0.7,1.1],brightness_range=[0.4,1.2])
train_batches= datagen.flow_from_directory(train_path, target_size=(targetsize,targetsize), shuffle=True,
                                                                      classes=['8', '5', '1', '7', '4', '0', '3', '6', '2', '9'],
                                                                      batch_size= batch_size_train)
test_batches= datagen.flow_from_directory(test_path,  target_size=(targetsize,targetsize), shuffle=True,
                                                                     classes=['8', '5', '1', '7', '4', '0', '3', '6', '2', '9'],
                                                                     batch_size= batch_size_train)

Found 6204 images belonging to 10 classes.
Found 1265 images belonging to 10 classes.


In [5]:
def plots(ims, figsize=(20,10), rows=1, interp= False, titles= None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims= ims.transpose((0,1,2,3))
    f= plt.figure(figsize=figsize)
    cols= len(ims)//rows if len(ims) %2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import model_from_json


In [7]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 5e-4 *0.99**x)

In [8]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))


model.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))

#model.add(Conv2D(filters=512, kernel_size=(3,3), padding='SAME', activation='relu'))
#model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 80, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 256)       2

In [9]:
model.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history2= model.fit_generator(train_batches, steps_per_epoch= len(train_batches) , 
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=500)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/500
125/125 [==============================] - 101s 811ms/step - loss: 2.2912 - accuracy: 0.1417 - val_loss: 2.2707 - val_accuracy: 0.1439
Epoch 2/500
125/125 [==============================] - 104s 829ms/step - loss: 2.2230 - accuracy: 0.1441 - val_loss: 2.2102 - val_accuracy: 0.1407
Epoch 3/500
125/125 [==============================] - 104s 834ms/step - loss: 2.2030 - accuracy: 0.1451 - val_loss: 2.1900 - val_accuracy: 0.1486
Epoch 4/500
125/125 [==============================] - 104s 834ms/step - loss: 2.1954 - accuracy: 0.1565 - val_loss: 2.2040 - val_accuracy: 0.1692
Epoch 5/500
125/125 [==============================] - 104s 829ms/step - loss: 2.1938 - accuracy: 0.1538 - val_loss: 2.1827 - val_accuracy: 0.1700
Epoch 6/500
125/125 [==============================] - 105s 841ms/step - loss: 2.1844 - accuracy: 0.1668 - val_loss: 2.1867 - val_accuracy: 0.1747
Epoch 7/500
125/125 [=====================

Epoch 55/500
125/125 [==============================] - 108s 864ms/step - loss: 1.8270 - accuracy: 0.3362 - val_loss: 1.8287 - val_accuracy: 0.3289
Epoch 56/500
125/125 [==============================] - 113s 902ms/step - loss: 1.8264 - accuracy: 0.3340 - val_loss: 1.8069 - val_accuracy: 0.3470
Epoch 57/500
125/125 [==============================] - 110s 882ms/step - loss: 1.8313 - accuracy: 0.3343 - val_loss: 1.7967 - val_accuracy: 0.3391
Epoch 58/500
125/125 [==============================] - 106s 850ms/step - loss: 1.7970 - accuracy: 0.3504 - val_loss: 1.7925 - val_accuracy: 0.3518
Epoch 59/500
125/125 [==============================] - 104s 831ms/step - loss: 1.8006 - accuracy: 0.3495 - val_loss: 1.8502 - val_accuracy: 0.3462
Epoch 60/500
125/125 [==============================] - 109s 874ms/step - loss: 1.8011 - accuracy: 0.3567 - val_loss: 1.8193 - val_accuracy: 0.3447
Epoch 61/500
125/125 [==============================] - 109s 868ms/step - loss: 1.7806 - accuracy: 0.3565 - val_

125/125 [==============================] - 135s 1s/step - loss: 1.3885 - accuracy: 0.5066 - val_loss: 1.4146 - val_accuracy: 0.5083
Epoch 167/500
125/125 [==============================] - 142s 1s/step - loss: 1.3919 - accuracy: 0.5132 - val_loss: 1.3314 - val_accuracy: 0.5225
Epoch 168/500
125/125 [==============================] - 145s 1s/step - loss: 1.4185 - accuracy: 0.5018 - val_loss: 1.4633 - val_accuracy: 0.4822
Epoch 169/500
125/125 [==============================] - 144s 1s/step - loss: 1.3935 - accuracy: 0.5069 - val_loss: 1.3189 - val_accuracy: 0.5399
Epoch 170/500
125/125 [==============================] - 136s 1s/step - loss: 1.4071 - accuracy: 0.5063 - val_loss: 1.3591 - val_accuracy: 0.5083
Epoch 171/500
125/125 [==============================] - 145s 1s/step - loss: 1.3917 - accuracy: 0.5073 - val_loss: 1.3940 - val_accuracy: 0.5012
Epoch 172/500
125/125 [==============================] - 144s 1s/step - loss: 1.4164 - accuracy: 0.4987 - val_loss: 1.3723 - val_accuracy:

125/125 [==============================] - 134s 1s/step - loss: 1.2505 - accuracy: 0.5754 - val_loss: 1.2546 - val_accuracy: 0.5826
Epoch 279/500
125/125 [==============================] - 143s 1s/step - loss: 1.1944 - accuracy: 0.5817 - val_loss: 1.2575 - val_accuracy: 0.5747
Epoch 280/500
125/125 [==============================] - 144s 1s/step - loss: 1.2106 - accuracy: 0.5853 - val_loss: 1.1831 - val_accuracy: 0.6016
Epoch 281/500
125/125 [==============================] - 140s 1s/step - loss: 1.2079 - accuracy: 0.5911 - val_loss: 1.2610 - val_accuracy: 0.5692
Epoch 282/500
125/125 [==============================] - 138s 1s/step - loss: 1.2282 - accuracy: 0.5758 - val_loss: 1.2622 - val_accuracy: 0.5771
Epoch 283/500
125/125 [==============================] - 144s 1s/step - loss: 1.2328 - accuracy: 0.5766 - val_loss: 1.2619 - val_accuracy: 0.5644
Epoch 284/500
125/125 [==============================] - 143s 1s/step - loss: 1.2483 - accuracy: 0.5803 - val_loss: 1.2758 - val_accuracy:

125/125 [==============================] - 133s 1s/step - loss: 1.0938 - accuracy: 0.6296 - val_loss: 1.1658 - val_accuracy: 0.6126
Epoch 391/500
125/125 [==============================] - 143s 1s/step - loss: 1.1497 - accuracy: 0.6190 - val_loss: 1.1005 - val_accuracy: 0.6403
Epoch 392/500
125/125 [==============================] - 143s 1s/step - loss: 1.1703 - accuracy: 0.6181 - val_loss: 1.0977 - val_accuracy: 0.6213
Epoch 393/500
125/125 [==============================] - 134s 1s/step - loss: 1.1732 - accuracy: 0.6120 - val_loss: 1.3213 - val_accuracy: 0.5929
Epoch 394/500
125/125 [==============================] - 142s 1s/step - loss: 1.1684 - accuracy: 0.6231 - val_loss: 1.1153 - val_accuracy: 0.6198
Epoch 395/500
125/125 [==============================] - 144s 1s/step - loss: 1.1345 - accuracy: 0.6310 - val_loss: 1.1026 - val_accuracy: 0.6379
Epoch 396/500
125/125 [==============================] - 141s 1s/step - loss: 1.0878 - accuracy: 0.6368 - val_loss: 1.2087 - val_accuracy:

In [10]:
model_json = model.to_json()
with open("model_1d10_final.json", "w") as json_file:
    json_file.write(model_json)

In [11]:
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

model.save('/home/ordovas/IRONHACK/dice-scores-recognition/model_d10_final.h5')